# Level 3 VQ-VAE Run Script

## Import a bunch of packages

In [ ]:
import IPython

import torch
from torch.utils.data import DataLoader
import yaml
import matplotlib.pyplot as plt
import torchaudio

from models.multi_level_vqvae import MultiLvlVQVariationalAutoEncoder
from loaders.music_loader import MP3SliceDataset
from loaders.lvl3_loader import Lvl3InputDataset
from utils.other import load_cfg_dict

## Load configuration files and add weights paths

In [ ]:
config_path_lvl1 = "config/lvl1_config.yaml"
weights_path_lvl1 = "model_weights/lvl1_vqvae.ckpt"
#weights_path = "model_best.ckpt"
cfg_1 = load_cfg_dict(config_path_lvl1)

config_path_lvl2 = "config/lvl2_config.yaml"
weights_path_lvl2 = "model_weights/lvl2_vqvae.ckpt"
#weights_path = "model_best.ckpt"
cfg_2 = load_cfg_dict(config_path_lvl2)

config_path_lvl3 = "config/lvl3_config.yaml"
weights_path_lvl3 = "model_weights/lvl3_vqvae.ckpt"
#weights_path = "model_best.ckpt"
cfg_3 = load_cfg_dict(config_path_lvl3)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Load the models

In [ ]:
model_lvl1 = MultiLvlVQVariationalAutoEncoder(**cfg_1).to(device)
model_lvl1 = model_lvl1.load_from_checkpoint(weights_path_lvl1, **cfg_1, strict=False).to(device)
model_lvl1.eval()

model_lvl2 = MultiLvlVQVariationalAutoEncoder(**cfg_2).to(device)
model_lvl2 = model_lvl1.load_from_checkpoint(weights_path_lvl2, **cfg_2, strict=False).to(device)
model_lvl2.eval()

model_lvl3 = MultiLvlVQVariationalAutoEncoder(**cfg_3).to(device)
model_lvl3 = model_lvl1.load_from_checkpoint(weights_path_lvl3, **cfg_3, strict=False).to(device)
model_lvl3.eval()

### HTML wrapper to display sound clips:

In [ ]:
# this is a wrapper that take a filename and publish an html <audio> tag to listen to it

def wavPlayer(filepath):
    """ will display html 5 player for compatible browser

    Parameters :
    ------------
    filepath : relative filepath with respect to the notebook directory ( where the .ipynb are not cwd)
               of the file to play

    The browser need to know how to play wav through html5.

    there is no autoplay to prevent file playing when the browser opens
    """
    
    src = """
    <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <title>Simple Test</title>
    </head>
    
    <body>
    <audio controls="controls" style="width:600px" >
      <source src="files/%s" type="audio/mp3" />
      Your browser does not support the audio element.
    </audio>
    </body>
    """%(filepath)
    display(HTML(src))

## Load a dataset sample and display it.

### Load the slice

In [ ]:
dataset = Lvl3InputDataset(preload=True)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
for sample in dataloader:
    lvl3_sample = sample['music slice'].squeeze(0)
    print(f"Current track is {sample['track name'][0]}")
    break
plt.figure(figsize=(25, 5))
plt.matshow(lvl3_sample.cpu().detach().numpy(), fignum=1, aspect='auto', vmin=-0.5, vmax=2.0)
plt.colorbar()
plt.axis('off')
plt.show()

## Run the level 3 network and show results

In [ ]:
lvl3_output = model_lvl3(lvl3_sample.clone().unsqueeze(0).to(device))
with torch.no_grad():
    lvl2_pred = lvl3_output['output'][0]
plt.figure(figsize=(25, 5))
plt.matshow(lvl2_pred.cpu().detach().numpy(), fignum=1, aspect='auto', vmin=-0.5, vmax=2.0)
plt.colorbar()
plt.axis('off')
plt.show()
print(lvl2_pred.size(), lvl3_sample.size())

## Show the subtraction between the ground truth and the reconstruction

In [ ]:
plt.figure(figsize=(25, 5))
plt.matshow(torch.abs(lvl3_sample.cpu() - lvl2_pred.cpu()).detach().numpy(), fignum=1, aspect='auto', vmin=0, vmax=2.0)
plt.colorbar()
plt.axis('off')
plt.show()

## Pass the level 2 outputs through the level 1 decoder and make MUSIC!

### Sanity Check, reconstruction of the lvl1 data

In [ ]:
batch_size = 8

with torch.no_grad():
    # z_q_out = vq_input
    z_q_out_lvl2 = model_lvl2.vq_module(lvl3_sample.unsqueeze(0).to(device))['v_q']
    output_lvl2 = model_lvl2.decoder(z_q_out_lvl2)
    output_lvl2 = output_lvl2.view((output_lvl2.shape[0] * batch_size, cfg_1['latent_depth'], -1))
    output_lvl1 = torch.zeros([1, cfg_1['input_channels'], 0]).to(device=device)
    
    for batch in output_lvl2:
        batch = batch.unsqueeze(0)
        z_q_out = model_lvl1.vq_module(batch.to(device))['v_q']
        output_part = model_lvl1.decoder(z_q_out)
        output_lvl1 = torch.cat((output_lvl1, output_part), dim=2)
    
music_sample_rec = output_lvl1.view((1, -1))
plt.figure(figsize=(25, 5))
plt.plot(music_sample_rec[0, ...].cpu().detach().numpy())
plt.ylim((-1.1, 1.1))
plt.show()
torchaudio.save('sample_out.mp3', music_sample_rec.cpu().detach(), 44100, format='mp3')
IPython.display.Audio(filename="sample_out.mp3")

### CREATE MUSIC!!!

In [ ]:
with torch.no_grad():
    # z_q_out = vq_input
    z_q_out_lvl2 = model_lvl2.vq_module(lvl2_pred.unsqueeze(0).to(device))['v_q']
    output_lvl2 = model_lvl2.decoder(z_q_out_lvl2)
    output_lvl2 = output_lvl2.view((output_lvl2.shape[0] * batch_size, cfg_1['latent_depth'], -1))
    output_lvl1 = torch.zeros([1, cfg_1['input_channels'], 0]).to(device=device)
    
    for batch in output_lvl2:
        batch = batch.unsqueeze(0)
        z_q_out = model_lvl1.vq_module(batch.to(device))['v_q']
        output_part = model_lvl1.decoder(z_q_out)
        output_lvl1 = torch.cat((output_lvl1, output_part), dim=2)
    
music_sample_rec = output_lvl1.view((1, -1))
plt.figure(figsize=(25, 5))
plt.plot(music_sample_rec[0, ...].cpu().detach().numpy())
plt.ylim((-1.1, 1.1))
plt.show()
torchaudio.save('sample_out.mp3', music_sample_rec.cpu().detach(), 44100, format='mp3')
IPython.display.Audio(filename="sample_out.mp3")